In [ ]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from matplotlib.colors import hsv_to_rgb

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

1. Write a program to *segment the boat and the people on it from the background*. Follow the instruction below:
    - Use 'boat.jpg' as input.
    - Apply Otsu thresholding.
    - Draw bounding box to identify the region where the boat and people are located.

In [ ]:
img = cv.imread("boat.jpg", 0)

blur = cv.GaussianBlur(img, (5, 5), 0)
ret, imgotsu = cv.threshold(blur, 0, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

result = img.copy()
contours = cv.findContours(imgotsu, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]
for cntr in contours:
    x,y,w,h = cv.boundingRect(cntr)
    cv.rectangle(result, (x, y), (x+w, y+h), (0, 0, 255), 2)
    #print("x,y,w,h:",x,y,w,h)

cv.imshow("boat", result)
cv.waitKey(0)
cv.destroyAllWindows()

2. Write a program to visualize the effect of size of structuring element on the morphological transformation (e.g. erosion, dilation, opening, and closing). 
    - Load the 'leaf.jfif' and perform thresholding to segment the leaf from the background.
    - Create a GUI containing trackbar to control the size of structuring element and display the resulting image on a window alongside the trackbar.

In [ ]:
imgleaf = cv.imread("leaf.jfif", 0)

blur = cv.GaussianBlur(imgleaf, (5, 5), 0)
ret, imgotsu = cv.threshold(blur, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)

def trackbar_erosion(x):
    kernel = np.ones((x, x), np.uint8)
    erosion = cv.erode(imgotsu, kernel, iterations = 1) #erosion
    dilation = cv.dilate(imgotsu, kernel,iterations = 1) #dilation
    opening = cv.morphologyEx(imgotsu, cv.MORPH_OPEN, kernel) #opening
    closing = cv.morphologyEx(imgotsu, cv.MORPH_CLOSE, kernel) # closing
    combined_images = np.hstack((imgotsu, erosion, dilation, opening, closing))
    
    cv.imshow('erosion', combined_images)

cv.namedWindow('erosion')
cv.createTrackbar('size(n)', 'erosion', 10, 20, trackbar_erosion)

cv.waitKey(0)
cv.destroyAllWindows()

3. **Apply the steps shown in exercise 4** to segment the porcelain from these images: "porcelain1.jfif" - "porcelain5.jfif".

In [ ]:
img = cv.imread('porcelain1.jfif')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

In [ ]:
r, g, b = cv.split(img)
fig = plt.figure()
axis = fig.add_subplot(1, 1, 1, projection = "3d")

pixel_colors = img.reshape((img.shape[0]*img.shape[1], 3))
norm = colors.Normalize(vmin = 0., vmax=1.)
norm.autoscale(pixel_colors)
pixel_colors = norm(pixel_colors).tolist()

axis.scatter(b.flatten(), g.flatten(), r.flatten(), facecolors = pixel_colors,
            marker = ".")
axis.set_xlabel('Blue')
axis.set_ylabel('Green')
axis.set_zlabel('Red')
plt.show()

In [ ]:
img_hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)

h, s, v = cv.split(img_hsv)
fig = plt.figure()
axis = fig.add_subplot(1, 1, 1, projection = "3d")

axis.scatter(h.flatten(), s.flatten(), v.flatten(), facecolors = pixel_colors,
            marker = ".")
axis.set_xlabel("Hue")
axis.set_ylabel("Saturation")
axis.set_zlabel("Value")
plt.show()

In [ ]:
blue_low = (80, 130, 20)
blue_high = (140, 255, 120)

from matplotlib.colors import hsv_to_rgb

lo_square = np.full((10, 10, 3), blue_low, dtype = np.uint8)/255.0
do_square = np.full((10, 10, 3), blue_high, dtype = np.uint8)/255.0

plt.subplot(1, 2, 1)
plt.imshow(hsv_to_rgb(do_square))
plt.subplot(1,2,2)
plt.imshow(hsv_to_rgb(lo_square))
plt.show()

In [ ]:
mask = cv.inRange(img_hsv, blue_low, blue_high)
res = cv.bitwise_and(img, img, mask = mask)

plt.subplot(1,2,1)
plt.imshow(mask, cmap = 'gray')
plt.subplot(1,2,2)
plt.imshow(res)
plt.show()

In [ ]:
white_low = (0, 0, 150)
white_high = (150, 45, 255)

lo_square = np.full((10, 10, 3), white_low, dtype = np.uint8)/255.0
do_square = np.full((10, 10, 3), white_high, dtype = np.uint8)/255.0

plt.subplot(1, 2, 1)
plt.imshow(hsv_to_rgb(do_square))
plt.subplot(1,2,2)
plt.imshow(hsv_to_rgb(lo_square))
plt.show()

In [ ]:
mask_white = cv.inRange(img_hsv, white_low, white_high)
res_white = cv.bitwise_and(img, img, mask = mask_white)

plt.subplot(1, 2, 1)
plt.imshow(mask_white, cmap = "gray")
plt.subplot(1, 2, 2)
plt.imshow(res_white)
plt.show()

In [ ]:
final_mask = cv.add(mask, mask_white)

In [ ]:
# Create a custom function
def segment_porcelain(img):
    """Image segmentation based on HSV color space to segregate
    blue and white regions from the provided argument img.
    img should be in RGB format.
    """
    # Convert image to HSV
    hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)

    # Set the blue color range in HSV color space
    blue_low = (80, 130, 20)
    blue_high = (140, 255, 120)

    # Create blue mask
    mask_blue = cv.inRange(hsv, blue_low, blue_high)

    # Set the white color range in HSV color space
    white_low = (0, 0, 150)
    white_high = (150, 45, 255)

    # Create white mask
    mask_white = cv.inRange(hsv, white_low, white_high)

    # AND operation to get the segmented blue region
    mask = cv.add(mask_blue, mask_white)
    res = cv.bitwise_and(img, img, mask = mask)

    # clean up the segmentation using blur filter
    blur = cv.GaussianBlur(res, (5, 5), 0)
    return blur
    
filename = "porcelain"

porcelain_list = []
for i in range(1,6):
    porcelain = cv.cvtColor(cv.imread(filename + str(i) + ".jfif"),
                           cv.COLOR_BGR2RGB)
    porcelain_list.append(porcelain)

results = [segment_porcelain(porcelain) for porcelain in porcelain_list]

In [ ]:
f, ax = plt.subplots(5,2, figsize = (18, 8))

for i in range(5):
    ax[i][0].imshow(porcelain_list[i])
    ax[i][1].imshow(results[i])
    
f.suptitle('Original images and their segmentation results')
plt.show()